In [8]:
import pandas as pd

pd.read_csv("/Users/ericyamnovski/Desktop/Codes/Project/llm-res/LLM-Social-Simulation/Log/September 07, 10 27PM , 2023Experimentlog.csv")



,attributes,pending_action,current_action_type,robbing_stats,obey_stats,memory,DESIRE_FOR_GLORY,DESIRE_FOR_PEACE,attributes.1,pending_action.1,...,DESIRE_FOR_GLORY.3,DESIRE_FOR_PEACE.3,attributes.4,pending_action.4,current_action_type.4,robbing_stats.4,obey_stats.4,memory.4,DESIRE_FOR_GLORY.4,DESIRE_FOR_PEACE.4
0,"{'id': 0, 'name': 'person 0', 'aggressiveness'...",<Main.Individual.SeralizeQueue object at 0x117...,farm,<Main.Individual.RobStats object at 0x117fa4400>,<Main.Individual.ObeyStats object at 0x132265c30>,"['None', 'None', 'None', 'None', 'None', 'None...",10,3,"{'id': 1, 'name': 'person 1', 'aggressiveness'...",<Main.Individual.SeralizeQueue object at 0x132...,...,10,3,"{'id': 4, 'name': 'person 4', 'aggressiveness'...",<Main.Individual.SeralizeQueue object at 0x132...,farm,<Main.Individual.RobStats object at 0x132266470>,<Main.Individual.ObeyStats object at 0x132266500>,"['None', 'None', 'None', 'None', 'None', 'None...",10,3
1,"{'id': 0, 'name': 'person 0', 'aggressiveness'...",<Main.Individual.SeralizeQueue object at 0x117...,rob,<Main.Individual.RobStats object at 0x117fa4400>,<Main.Individual.ObeyStats object at 0x132265c30>,"['None', 'None', 'None', 'None', 'None', 'None...",10,3,"{'id': 1, 'name': 'person 1', 'aggressiveness'...",<Main.Individual.SeralizeQueue object at 0x132...,...,10,3,"{'id': 4, 'name': 'person 4', 'aggressiveness'...",<Main.Individual.SeralizeQueue object at 0x132...,rob,<Main.Individual.RobStats object at 0x132266470>,<Main.Individual.ObeyStats object at 0x132266500>,"['None', 'None', 'None', 'None', 'None', 'None...",10,3
2,"{'id': 0, 'name': 'person 0', 'aggressiveness'...",<Main.Individual.SeralizeQueue object at 0x117...,farm,<Main.Individual.RobStats object at 0x117fa4400>,<Main.Individual.ObeyStats object at 0x132265c30>,"['None', 'None', 'None', 'None', 'None', 'None...",10,3,"{'id': 1, 'name': 'person 1', 'aggressiveness'...",<Main.Individual.SeralizeQueue object at 0x132...,...,10,3,"{'id': 4, 'name': 'person 4', 'aggressiveness'...",<Main.Individual.SeralizeQueue object at 0x132...,farm,<Main.Individual.RobStats object at 0x132266470>,<Main.Individual.ObeyStats object at 0x132266500>,"['None', 'None', 'None', 'None', 'None', 'None...",10,3
3,"{'id': 0, 'name': 'person 0', 'aggressiveness'...",<Main.Individual.SeralizeQueue object at 0x117...,farm,<Main.Individual.RobStats object at 0x117fa4400>,<Main.Individual.ObeyStats object at 0x132265c30>,"['None', 'None', 'None', 'None', 'None', 'None...",10,3,"{'id': 1, 'name': 'person 1', 'aggressiveness'...",<Main.Individual.SeralizeQueue object at 0x132...,...,10,3,"{'id': 4, 'name': 'person 4', 'aggressiveness'...",<Main.Individual.SeralizeQueue object at 0x132...,farm,<Main.Individual.RobStats object at 0x132266470>,<Main.Individual.ObeyStats object at 0x132266500>,"['None', 'None', 'None', 'None', 'None', 'None...",10,3
4,"{'id': 0, 'name': 'person 0', 'aggressiveness'...",<Main.Individual.SeralizeQueue object at 0x117...,be robbed,<Main.Individual.RobStats object at 0x117fa4400>,<Main.Individual.ObeyStats object at 0x132265c30>,"['None', 'None', 'None', 'None', 'None', 'None...",10,3,"{'id': 1, 'name': 'person 1', 'aggressiveness'...",<Main.Individual.SeralizeQueue object at 0x132...,...,10,3,"{'id': 4, 'name': 'person 4', 'aggressiveness'...",<Main.Individual.SeralizeQueue object at 0x132...,farm,<Main.Individual.RobStats object at 0x132266470>,<Main.Individual.ObeyStats object at 0x132266500>,"['None', 'None', 'None', 'None', 'None', 'None...",10,3
5,"{'id': 0, 'name': 'person 0', 'aggressiveness'...",<Main.Individual.SeralizeQueue object at 0x117...,be robbed,<Main.Individual.RobStats object at 0x117fa4400>,<Main.Individual.ObeyStats object at 0x132265c30>,"['None', 'None', 'None', 'None', 'None', 'None...",10,3,"{'id': 1, 'name': 'person 1', 'aggressiveness'...",<Main.Individual.SeralizeQueue object at 0x132...,...,10,3,"{'id': 4, 'name': 'person 4', 'aggressiveness'...",<Main.Individual.SeralizeQueue object at 0x132...,farm,<